In [2]:
import numpy as np
import pandas as pd

# load the sample data to evaluate the model
df = pd.read_csv("../data/GE.csv")

# add new column Open_before which contains the open values of the previous day
df["Open_before"] = df["Open"].shift(1)

# calculate the procentual change of the open value of the current day to the 
# open value of the day before
df["Open_changes"] = (df["Open"] / df["Open_before"]) - 1

# throw out the first line which has NaN as value because of the previous shift of values 
df = df.dropna()

# reset index to start by 0
df.index -= 1

changes = df["Open_changes"]

In [4]:
from sklearn.preprocessing import MinMaxScaler
# normalize data

# border to split in test and training data
split_border = int(len(changes) * 0.8)

# build test and training data
train = np.array(changes[:split_border]).reshape(-1, 1)
test = np.array(changes[split_border:]).reshape(-1, 1)

print(train.shape)
print(train)

scaler = MinMaxScaler()

# train the scaler with training data and smooth data
smoothing_window_size = 2500
for di in range(0, 10000, smoothing_window_size):
    scaler.fit(train[di:di+smoothing_window_size,:])
    train[di:di+smoothing_window_size,:] = scaler.transform(train[di:di+smoothing_window_size,:])

# normalize the rest of the data which is len(train) - 10000
scaler.fit(train[di+smoothing_window_size:,:])
train[di+smoothing_window_size:,:] = scaler.transform(train[di+smoothing_window_size:,:])

# normalize test data
test = scaler.transform(test)

# reshape test and train data
train = train.reshape(-1)
test = test.reshape(-1)

(11614, 1)
[[-0.00833331]
 [-0.005042  ]
 [-0.01182429]
 ...
 [-0.01475702]
 [ 0.00910426]
 [-0.01891735]]


In [4]:
# perform exponential moving average
EMA = 0.0
gamma = 0.1
for ti in range(len(train)):
    EMA = gamma * train[ti] + (1 - gamma) * EMA
    train[ti] = EMA

In [6]:
# TODO: Wie kriege ich aus den normalisierten Daten nun meine X und Y Daten im richtigen Format
train_len = len(train)
test_len = len(test)

Y_train = np.array(train[:train_len-20])
Y_test = np.array(test[:test_len-20])
X_train = []
X_test = []

# predict price with previous 20 entries
for i in range(0, len(train) - 20):
    X_train.append(np.array(train[i+1:i+21]))
for i in range(0, len(test) - 20):
    X_test.append(np.array(test[i+1:i+21]))

# reshape training data
X_train = np.array(X_train).reshape(-1, 20, 1)
X_test = np.array(X_test).reshape(-1, 20, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(20, 1), return_sequences=True))
model.add(LSTM(64, input_shape=(20,1)))

model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X_train, Y_train, batch_size=32, epochs=10)

model.save('../models/multiple-layer-lstm-with-dense-with-normalization.h5')

Using TensorFlow backend.
W0907 14:27:29.034299 10588 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0907 14:27:29.072890 10588 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0907 14:27:29.075883 10588 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0907 14:27:29.650713 10588 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0907 

Epoch 1/10
11594/11594 [==============================] - 18s 2ms/step - loss: 0.0017
Epoch 2/10
11594/11594 [==============================] - 15s 1ms/step - loss: 5.7427e-04
Epoch 3/10
11594/11594 [==============================] - 15s 1ms/step - loss: 4.4054e-04
Epoch 4/10
11594/11594 [==============================] - 16s 1ms/step - loss: 3.9581e-04
Epoch 5/10
 4928/11594 [===========>..................] - ETA: 8s - loss: 3.4406e-04

In [ ]:
# Faktor um den das eingesetzte Vermögen steigt aktueller Stand / damaliger Stand 
# Berechne Rendite für Testdaten
# An der Stelle split_border ist df["Open_changes"] gleich Y_test[0]
rendite = df["Open"][14517] / df["Open"][split_border]

# Marktrendite bei 100 Euro Einsatz
markt_return = 100 * rendite

# Rendite bei Trading durch das Modell
buy_or_sell = []

# predictions of Neuronal Net
predictions = model.predict(X_test)

predictions = np.array(predictions).reshape(-1)

# True when buy stock, false when sell stock
for change in predictions: 
    if change > 0:
        buy_or_sell.append(True)
    else:
        buy_or_sell.append(False)

In [ ]:
# Berechne Rendite bei Neuronalem Netz
# 1. Iteriere über buy_or_sell
# 2. Wenn prediction True dann kaufe Aktie zu diesem Zeitpunkt 
# 3. Wenn prediction False dann verkaufe alle Aktien

# starte mit kapital von 100 Euro
kapital = 100
# Anzahl Aktien im Depot
depot = 0
# Kurs in Euro pro Aktie
kurs = 0

# Annahme: Investiere immer das komplette Kapital bzw verkaufe komplettes Depot
for (prediction, i) in zip(buy_or_sell, range(split_border, 14518 - 20)):
    if prediction == True and kapital > 0:
        # kaufe Aktie zum Zeitpunkt i
        kurs = df["Open"][i]
        # depot[Aktien] = Kapital[Euro] * (1 [Aktie] / Kurs [Euro pro Aktie])
        depot += kapital * (1 / kurs)
        kapital = 0
    elif depot > 0:
        # verkaufe Aktien im Depot zum Zeitpunkt i
        kurs = df["Open"][i]
        kapital = depot * kurs 
        depot = 0

print(kapital)
print(markt_return)